In [1]:
import re
import string
import pandas as pd
import numpy as np
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
pcsvpath = '../pollen/datafile.csv'
sample = pd.read_csv(pcsvpath)

In [3]:
sample

,date,lat,lon,t,2t
0,2000-01-01,-10.0,113.0,302.2,300.6
1,2000-01-01,-10.0,114.0,302.2,300.8
2,2000-01-01,-10.0,115.0,302.2,301.3
3,2000-01-01,-10.0,116.0,302.4,301.9
4,2000-01-01,-10.0,117.0,302.5,302.3
...,...,...,...,...,...
177067,2000-01-31 18:00:00,-43.0,150.0,289.4,287.9
177068,2000-01-31 18:00:00,-43.0,151.0,289.1,287.3
177069,2000-01-31 18:00:00,-43.0,152.0,288.9,287.4
177070,2000-01-31 18:00:00,-43.0,153.0,288.6,287.4


t : surface level temp </br>
2t: temp at 2m level </br>

In [4]:
sample.dtypes

date     object
lat     float64
lon     float64
t       float64
2t      float64
dtype: object

## Preprocessing

In [5]:
def kelvinToCelsius(kelvin):
    
    return kelvin - 273.15

### 1. Filter Melbourne 
Coord for Melbourne: -37.814, 144.96332

In [6]:
# consider the grid

melbourne = [-37.814, 144.96332]
lat = float(round(melbourne[0])) # -38
lon = float(round(melbourne[1])) # 145

sample_mel = sample[(sample['lat'] == lat) & (sample['lon'] == lon)]

### 2. Convert unit

In [7]:
# convert unit
sample_mel['t_C'] = sample_mel['t'].apply(kelvinToCelsius)
sample_mel['2t_C'] = sample_mel['2t'].apply(kelvinToCelsius)

### 3. Datetime

In [8]:
## utc->aest

sample_mel['datetime'] = pd.to_datetime(sample_mel['date'], format='%Y-%m-%d %H:%M:%S')

sample_mel = sample_mel.drop(['date', 't', '2t', 'lat', 'lon'], axis=1)

sample_mel

,t_C,2t_C,datetime
1208,15.450000,15.15,2000-01-01 00:00:00
2636,19.150000,18.25,2000-01-02 00:00:00
4064,25.150000,24.75,2000-01-03 00:00:00
5492,11.350000,11.45,2000-01-04 00:00:00
6920,11.250000,11.35,2000-01-05 00:00:00
...,...,...,...
171140,15.050000,13.25,2000-01-27 18:00:00
172568,14.550024,12.95,2000-01-28 18:00:00
173996,14.250000,12.25,2000-01-29 18:00:00
175424,13.450000,11.15,2000-01-30 18:00:00


In [9]:
from dateutil import tz

# METHOD 2: Auto-detect zones:
from_zone = tz.tzutc()
to_zone = tz.tzlocal()

# utc = datetime.utcnow()
utc = datetime.strptime('2000-01-31 18:00:00', '%Y-%m-%d %H:%M:%S')

# Tell the datetime object that it's in UTC time zone since 
# datetime objects are 'naive' by default
utc = utc.replace(tzinfo=from_zone)

# Convert time zone
local = utc.astimezone(to_zone).strftime('%Y-%m-%d %H:%M:%S')

print(local)

2000-02-01 05:00:00


## handle with different features

In [10]:
df_new = sample_mel.copy()

# extract date
df_new['date'] = df_new['datetime'].dt.strftime('%Y-%m-%d')

# extract hour
df_new['hour'] = df_new['datetime'].dt.hour

# print out four hour
df_new['hour'].value_counts().index.to_list()

[0, 6, 12, 18]

In [11]:
# df_new[df_new['date'] == '2000-01-01']

### Create new columns for 4 hours

In [12]:
def new_hour_df(df, hour):

    data_h_new = df[df['hour'] == hour]

    data_h_new = data_h_new.set_index('date')   

    data_h_new.rename(columns = {'t_C':'t_' + str(hour) + '_1d', '2t_C':'2t_' + str(hour) + '_1d'}, inplace = True)
    
    data_h_new = data_h_new.drop(['hour', 'datetime'], axis=1)
    
    return data_h_new

In [13]:
df_0 = new_hour_df(df_new, 0)
df_6 = new_hour_df(df_new, 6)
df_12 = new_hour_df(df_new, 12)
df_18 = new_hour_df(df_new, 18)

In [14]:
df_12

,t_12_1d,2t_12_1d
date,,
2000-01-01,10.85,10.85
2000-01-02,15.65,15.55
2000-01-03,13.95,13.45
2000-01-04,8.85,9.05
2000-01-05,10.25,10.45
2000-01-06,10.95,11.05
2000-01-07,13.85,13.95
2000-01-08,19.85,19.75
2000-01-09,21.85,21.55


### Calculate the mean/min/max of 4 hours for one day

In [15]:
# group by date with hours by mean
df_mean = df_new.groupby('date')[['t_C', '2t_C']].mean()

df_mean.rename(columns = {'t_C':'t_mean_1d', '2t_C':'2t_mean_1d'}, inplace = True)

df_mean

,t_mean_1d,2t_mean_1d
date,,
2000-01-01,12.725000,12.600
2000-01-02,18.575000,18.275
2000-01-03,18.450000,18.150
2000-01-04,10.425000,10.525
2000-01-05,11.150000,11.200
2000-01-06,12.875000,12.800
2000-01-07,15.350000,15.125
2000-01-08,21.325000,20.950
2000-01-09,23.850006,23.525


In [16]:
# group by date with hours by min
df_min = df_new.groupby('date')[['t_C', '2t_C']].min()

df_min.rename(columns = {'t_C':'t_min_1d', '2t_C':'2t_min_1d'}, inplace = True)

df_min

,t_min_1d,2t_min_1d
date,,
2000-01-01,6.950000,7.25
2000-01-02,14.650000,14.95
2000-01-03,9.350000,9.45
2000-01-04,8.850000,9.05
2000-01-05,9.850000,9.95
2000-01-06,9.850000,9.95
2000-01-07,10.050000,10.05
2000-01-08,16.050000,16.05
2000-01-09,19.550000,19.45


In [17]:
# group by date with hours by max
df_max = df_new.groupby('date')[['t_C', '2t_C']].max()

df_max.rename(columns = {'t_C':'t_max_1d', '2t_C':'2t_max_1d'}, inplace = True)

df_max

,t_max_1d,2t_max_1d
date,,
2000-01-01,17.65,17.15
2000-01-02,24.85,24.35
2000-01-03,25.35,24.95
2000-01-04,12.55,12.35
2000-01-05,13.25,13.05
2000-01-06,15.85,15.65
2000-01-07,20.45,19.95
2000-01-08,27.05,26.55
2000-01-09,28.25,27.85


In [18]:
df = df_0.join(df_6).join(df_12).join(df_18).join(df_mean).join(df_min).join(df_max)
df

,t_0_1d,2t_0_1d,t_6_1d,2t_6_1d,t_12_1d,2t_12_1d,t_18_1d,2t_18_1d,t_mean_1d,2t_mean_1d,t_min_1d,2t_min_1d,t_max_1d,2t_max_1d
date,,,,,,,,,,,,,,
2000-01-01,15.450000,15.15,17.65,17.15,10.85,10.85,6.950000,7.25,12.725000,12.600,6.950000,7.25,17.65,17.15
2000-01-02,19.150000,18.25,24.85,24.35,15.65,15.55,14.650000,14.95,18.575000,18.275,14.650000,14.95,24.85,24.35
2000-01-03,25.150000,24.75,25.35,24.95,13.95,13.45,9.350000,9.45,18.450000,18.150,9.350000,9.45,25.35,24.95
2000-01-04,11.350000,11.45,12.55,12.35,8.85,9.05,8.950000,9.25,10.425000,10.525,8.850000,9.05,12.55,12.35
2000-01-05,11.250000,11.35,13.25,13.05,10.25,10.45,9.850000,9.95,11.150000,11.200,9.850000,9.95,13.25,13.05
2000-01-06,14.850000,14.55,15.85,15.65,10.95,11.05,9.850000,9.95,12.875000,12.800,9.850000,9.95,15.85,15.65
2000-01-07,17.050000,16.55,20.45,19.95,13.85,13.95,10.050000,10.05,15.350000,15.125,10.050000,10.05,20.45,19.95
2000-01-08,22.350000,21.45,27.05,26.55,19.85,19.75,16.050000,16.05,21.325000,20.950,16.050000,16.05,27.05,26.55
2000-01-09,25.750024,25.25,28.25,27.85,21.85,21.55,19.550000,19.45,23.850006,23.525,19.550000,19.45,28.25,27.85


In [19]:
# making variables for t: surface level temp

#df['t_mean_3d'] = df['t_mean_1d'].transform(lambda x: x.rolling(3, center=False).mean())
#df['t_max_3d'] = df['t_max_1d'].transform(lambda x: x.rolling(3, center=False).max())
#df['t_min_3d'] = df['t_min_1d'].transform(lambda x: x.rolling(3, center=False).min())

In [20]:
# check t_mean_3d: (12.725+18.575+18.450)/3 = 16.583333333333332 -> 2000-01-03

In [21]:
# making variables for 2t: temp at 2m level

#df['2t_mean_3d'] = df['2t_mean_1d'].transform(lambda x: x.rolling(3, center=False).mean())
#df['2t_max_3d'] = df['2t_max_1d'].transform(lambda x: x.rolling(3, center=False).max())
#df['2t_min_3d'] = df['2t_min_1d'].transform(lambda x: x.rolling(3, center=False).min())

In [22]:
# write in function 

def make_variables_past_days(var_list, day_list, df):
    
    for var in var_list:
        
        for i in day_list:
        
            df[var + '_mean_' + str(i) + 'd'] = df[var + '_mean_1d'].transform(lambda x: x.rolling(i, center=False).mean())
            df[var + '_max_' + str(i) + 'd'] = df[var + '_max_1d'].transform(lambda x: x.rolling(i, center=False).max())
            df[var + '_min_' + str(i) + 'd'] = df[var + '_min_1d'].transform(lambda x: x.rolling(i, center=False).min())
        
    return df

In [23]:
# displaying the DataFrame
display(make_variables_past_days(['t', '2t'], [3,5,7,30], df))

,t_0_1d,2t_0_1d,t_6_1d,2t_6_1d,t_12_1d,2t_12_1d,t_18_1d,2t_18_1d,t_mean_1d,2t_mean_1d,...,2t_min_3d,2t_mean_5d,2t_max_5d,2t_min_5d,2t_mean_7d,2t_max_7d,2t_min_7d,2t_mean_30d,2t_max_30d,2t_min_30d
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,15.450000,15.15,17.65,17.15,10.85,10.85,6.950000,7.25,12.725000,12.600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,19.150000,18.25,24.85,24.35,15.65,15.55,14.650000,14.95,18.575000,18.275,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,25.150000,24.75,25.35,24.95,13.95,13.45,9.350000,9.45,18.450000,18.150,...,7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,11.350000,11.45,12.55,12.35,8.85,9.05,8.950000,9.25,10.425000,10.525,...,9.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,11.250000,11.35,13.25,13.05,10.25,10.45,9.850000,9.95,11.150000,11.200,...,9.05,14.150,24.95,7.25,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,14.850000,14.55,15.85,15.65,10.95,11.05,9.850000,9.95,12.875000,12.800,...,9.05,14.190,24.95,9.05,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,17.050000,16.55,20.45,19.95,13.85,13.95,10.050000,10.05,15.350000,15.125,...,9.95,13.560,24.95,9.05,14.096429,24.95,7.25,NaN,NaN,NaN
2000-01-08,22.350000,21.45,27.05,26.55,19.85,19.75,16.050000,16.05,21.325000,20.950,...,9.95,14.120,26.55,9.05,15.289286,26.55,9.05,NaN,NaN,NaN
2000-01-09,25.750024,25.25,28.25,27.85,21.85,21.55,19.550000,19.45,23.850006,23.525,...,10.05,16.720,27.85,9.95,16.039286,27.85,9.05,NaN,NaN,NaN


In [24]:
df.shape

(31, 38)